In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
vehicle_dataset = torchvision.datasets.CIFAR10(root="./data", train=True, transform=transform, download=True)
vehicle_loader = torch.utils.data.DataLoader(vehicle_dataset, batch_size=100, shuffle=True)

# Define the MLP
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 500)
        self.fc2 = nn.Linear(500, 100)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        x = x.view(-1, 32 * 32 * 3)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function, and optimizer
model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Train the MLP
for epoch in range(10):
    running_loss = 0.0
    for images, labels in vehicle_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print("Epoch [{}/10], Loss: {:.4f}".format(epoch + 1, running_loss / len(vehicle_loader)))

# Save the model
torch.save(model.state_dict(), "vehicle_mlp.pt")
